In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *
from sklearn.neighbors import KDTree
from lrg_plot_functions import *
from lrg_sum_functions import *
from cosmo_Calc import *
from divideByTwo import *
from read_data import *
from nearNeighbors import *
from scipy import stats
from astropy import stats
import healpy as hp
from localBKG_and_interlopersHEALPix import *
# import healpy as hpy
from makeCutoutTableFunc import *
from plotHistogramProjections3D import *
from read_bricks import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
from sigSats import *
from sigSats_usingSumsat import *
from confidence_interval import *


In [3]:
# read in null data

# lowz
low_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/low_Nsat_null-EDR.npz')
low_Nsat_total = low_data_Nsat['Nsat']
low_Nsat35 = low_data_Nsat['low_nullNsat35']

print(low_Nsat_total.shape)
print(low_Nsat35.shape)

# midz
mid_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/mid_Nsat_null-EDR.npz')
mid_Nsat_total = mid_data_Nsat['Nsat']
mid_Nsat35 = mid_data_Nsat['mid_nullNsat35']
mid_Nsat05 = mid_data_Nsat['mid_nullNsat05']

print(mid_Nsat_total.shape)
print(mid_Nsat35.shape)
print(mid_Nsat05.shape)

# highz
high_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/high_Nsat_null-EDR.npz')
high_Nsat_total = high_data_Nsat['Nsat']
high_Nsat35 = high_data_Nsat['high_nullNsat35']
high_Nsat05 = high_data_Nsat['high_nullNsat05']
high_Nsat65 = high_data_Nsat['high_nullNsat65']

print(high_Nsat_total.shape)
print(high_Nsat35.shape)
print(high_Nsat05.shape)
print(high_Nsat65.shape)



(5000, 49, 49, 49)
(5000, 49, 49, 49)
(5000, 49, 49, 49)
(1689, 49, 49, 49)
(3311, 49, 49, 49)
(5000, 49, 49, 49)
(862, 49, 49, 49)
(1721, 49, 49, 49)
(2417, 49, 49, 49)


In [4]:
# read in selection matrices

# lowz
smatrix_low = np.load('/Users/mtownsend/Desktop/lowz_smatrix-1matrices-EDR.npz')
low_smatrix = smatrix_low['smatrix']
smatrix_low35 = low_smatrix[0]
print(smatrix_low35.shape)

print()
print()

# midz
smatrix_mid = np.load('/Users/mtownsend/Desktop/midz_smatrix-2matrices-EDR.npz')
mid_smatrix = smatrix_mid['smatrix']
smatrix_mid35 = mid_smatrix[0]
smatrix_mid05 = mid_smatrix[1]
print(smatrix_mid35.shape)
print(smatrix_mid05.shape)

print()
print()

# highz
smatrix_high = np.load('/Users/mtownsend/Desktop/highz_smatrix-3matrices-EDR.npz')
high_smatrix = smatrix_high['smatrix']
smatrix_high35 = high_smatrix[0]
smatrix_high05 = high_smatrix[1]
smatrix_high65 = high_smatrix[2]
print(smatrix_high35.shape)
print(smatrix_high05.shape)
print(smatrix_high65.shape)

(49, 49, 49)


(49, 49, 49)
(49, 49, 49)


(49, 49, 49)
(49, 49, 49)
(49, 49, 49)


In [5]:
# apply masks

# lowz
masked_Nsat_low35 = []
for i in range(len(low_Nsat35)):
    masked_Nsat_low35.append(low_Nsat35[i] * smatrix_low35)

print(np.asarray(masked_Nsat_low35).shape)

# midz
masked_Nsat_mid35 = []
for i in range(len(mid_Nsat35)):
    masked_Nsat_mid35.append(mid_Nsat35[i] * smatrix_mid35)
    
print(np.asarray(masked_Nsat_mid35).shape)
    
masked_Nsat_mid05 = []
for i in range(len(mid_Nsat05)):
    masked_Nsat_mid05.append(mid_Nsat05[i] * smatrix_mid05)
    
print(np.asarray(masked_Nsat_mid05).shape)


# highz
masked_Nsat_high35 = []
for i in range(len(high_Nsat35)):
    masked_Nsat_high35.append(high_Nsat35[i] * smatrix_high35)
    
print(np.asarray(masked_Nsat_high35).shape)
    
masked_Nsat_high05 = []
for i in range(len(high_Nsat05)):
    masked_Nsat_high05.append(high_Nsat05[i] * smatrix_high05)
    
print(np.asarray(masked_Nsat_high05).shape)
    
masked_Nsat_high65 = []
for i in range(len(high_Nsat65)):
    masked_Nsat_high65.append(high_Nsat65[i] * smatrix_high65)
    
print(np.asarray(masked_Nsat_high65).shape)
    

(5000, 49, 49, 49)
(1689, 49, 49, 49)
(3311, 49, 49, 49)
(862, 49, 49, 49)
(1721, 49, 49, 49)
(2417, 49, 49, 49)


In [6]:
def totalNsat(Nsat, lum, z1, z2):

    import matplotlib.pylab as plt 
    import numpy as np
    from scipy import stats
    
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height


    sumsat = []

    # Sum up number of satellite galaxies for every LRG
    for i in range(len(Nsat)):
        sumsat.append(np.sum(Nsat[i]))

    meansat = np.mean(sumsat)
    print("mean number of satellites is", meansat)
    
    mediansat = np.median(sumsat)
    print("median number of satellites is", mediansat)

    sdsat = np.std(sumsat)
#     print("standard deviation of satellites is", sdsat)

    sterr = stats.sem(sumsat)
#     print("standard error is", sterr)
    
    eighty = np.percentile(sumsat, 80.)
    print("80th percentile = ", eighty)
    ninety = np.percentile(sumsat, 90.)
    print("90th percentile = ", ninety)
    ninetynine = np.percentile(sumsat, 99.)
    print("99th percentile = ", ninetynine)
    

#     plt.rcParams["figure.figsize"] = [15, 8]
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
# #     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)=10.36; 0.55 < z < 0.6", fontsize=25)
#     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)={}; {} < z < {}".format(lum, z1, z2), fontsize=20)
#     plt.hist(sumsat, bins=50)
#     plt.xlabel(r'$Number$ $of$ $Satellite$ $Galaxies$', fontsize=25)
#     plt.ylabel(r'$counts$', fontsize=25)
#     plt.axvline(x=eighty, linewidth=1, ls='-', color='r')
#     plt.axvline(x=ninety, linewidth=1, ls='--', color='g')
#     plt.axvline(x=ninetynine, linewidth=1, ls=':', color='b')
#     plt.text(top, right,'80th = {}'.format(eighty), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     plt.text(top, right-0.05,'90th = {}'.format(ninety), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     plt.text(top, right-0.1,'99th = {}'.format(ninetynine), fontsize=14, ha='center', va='center', transform=ax.transAxes)
    # plt.show()
    return(eighty, ninety, ninetynine)


In [7]:
zrange = [0.2, 0.35, 0.5, 0.65]

# low
low_p35 = totalNsat(masked_Nsat_low35, 9.27, round(zrange[0], 2), round(zrange[1], 2))

np.savez('/Users/mtownsend/Documents/LRG_Data/percentiles_low-EDR.npz', low_p35=low_p35)
data_low = np.load('/Users/mtownsend/Documents/LRG_Data/percentiles_low-EDR.npz')
p35 = data_low['low_p35']
print(p35)

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat25_5000.pdf')
# plt.show()

print('----')

# mid
mid_p35 = totalNsat(masked_Nsat_mid35, 9.58, round(zrange[0], 2), round(zrange[1], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

mid_p05 = totalNsat(masked_Nsat_mid05, 9.58, round(zrange[1], 2), round(zrange[2], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

np.savez('/Users/mtownsend/Documents/LRG_Data/percentiles_mid-EDR.npz', mid_p35=mid_p35, mid_p05=mid_p05)
data_mid = np.load('/Users/mtownsend/Documents/LRG_Data/percentiles_mid-EDR.npz')
p35 = data_mid['mid_p35']
p05 = data_mid['mid_p05']
print(p35)
print(p05)

print('----')

# high
high_p35 = totalNsat(masked_Nsat_high35, 9.85, round(zrange[0], 2), round(zrange[1], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

high_p05 = totalNsat(masked_Nsat_high05, 9.85, round(zrange[1], 2), round(zrange[2], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

high_p65 = totalNsat(masked_Nsat_high65, 9.85, round(zrange[2], 2), round(zrange[3], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

np.savez('/Users/mtownsend/Documents/LRG_Data/percentiles_high-EDR.npz', high_p35=high_p35, high_p05=high_p05, high_p65=high_p65)
data_high = np.load('/Users/mtownsend/Documents/LRG_Data/percentiles_high-EDR.npz')
p35 = data_high['high_p35']
p05 = data_high['high_p05']
p65 = data_high['high_p65']

print(p35)
print(p05)
print(p65)


mean number of satellites is 0.7338738396920366
median number of satellites is 0.12302222960133014
80th percentile =  3.60085828545981
90th percentile =  5.808602245498766
99th percentile =  13.518836077710104
[ 3.60085829  5.80860225 13.51883608]
----
mean number of satellites is 0.3295916687078241
median number of satellites is -0.13430074008918735
80th percentile =  2.1073963972043326
90th percentile =  3.6195333228577367
99th percentile =  8.925262531670223
mean number of satellites is 0.769620358765144
median number of satellites is 0.27712554795158517
80th percentile =  3.0767792691210736
90th percentile =  5.035104699110522
99th percentile =  11.990603456393144
[2.1073964  3.61953332 8.92526253]
[ 3.07677927  5.0351047  11.99060346]
----
mean number of satellites is 0.24962143660782768
median number of satellites is -0.06792544669847939
80th percentile =  1.2838840869839667
90th percentile =  2.1810592782677496
99th percentile =  6.12966590424827
mean number of satellites is 0.3

In [ ]:
# mean number of satellites is 0.43185995286792667
# median number of satellites is -0.07769604254230897
# 80th percentile =  2.450666873511643
# 90th percentile =  4.045633909675518
# 99th percentile =  9.706887147606526
# [2.45066687 4.04563391 9.70688715]
# ----
# mean number of satellites is 0.08866369903652377
# median number of satellites is -0.2448052275004367
# 80th percentile =  0.7330611419848321
# 90th percentile =  1.447594176457257
# 99th percentile =  3.9937562462791334
# mean number of satellites is 0.364447281362359
# median number of satellites is -0.03475007929818505
# 80th percentile =  1.7657694723228268
# 90th percentile =  2.951285431763983
# 99th percentile =  7.67940202145567
# [0.73306114 1.44759418 3.99375625]
# [1.76576947 2.95128543 7.67940202]
# ----
# mean number of satellites is 0.08079884443066397
# median number of satellites is -0.30284829092036875
# 80th percentile =  0.6299348187130149
# 90th percentile =  1.256018194225554
# 99th percentile =  2.8942652194677447
# mean number of satellites is 0.11170557814731073
# median number of satellites is -0.3037891690353054
# 80th percentile =  0.6562299046502136
# 90th percentile =  1.4233063651350735
# 99th percentile =  3.5495715513635546
# mean number of satellites is 0.30045002719758557
# median number of satellites is -0.014074679559070546
# 80th percentile =  1.5842319156646094
# 90th percentile =  2.777955092012864
# 99th percentile =  5.968297624976673
# [0.62993482 1.25601819 2.89426522]
# [0.6562299  1.42330637 3.54957155]
# [1.58423192 2.77795509 5.96829762]
